In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from mongo import attraction_db
from model.google import Google
from model.Spider import Spider 
from model.Word import Word 
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import csv
import re
import os

In [2]:
def sort_by_weekday(operating_hours):
    days_order = ['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日']
    sorted_hours = sorted(operating_hours, key=lambda x: days_order.index(x['week']))
    return sorted_hours

def scrollPage(driver,times):   # 滾動頁面
    counter = 0
    while counter <= times:
        pane = driver.find_element("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]')
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
        time.sleep(1)
        counter += 1

def get_place_more_info(driver):
    try:
        data={}
        page_content = driver.page_source
        soup = BeautifulSoup(page_content, 'html.parser')
        open_time_week = []
        try:data["subtitle"] = soup.select('.DkEaL')[0].text
        except:data["subtitle"] = ""
        try:data["info"] = soup.select('.PYvSYb')[0].text
        except:data["info"] = ""
        try:data["site"] = soup.select('a.CsEnBe')[0].get('href')
        except:data["site"] = ""
        try:
            open_time = soup.select('.y0skZc')
            for time_up in open_time:
                week = time_up.select('td div')[0].text
                clock = time_up.select('td.mxowUb .G8aQO')[0].text
                result={}
                result['week'] = week
                result['clock'] = clock
                open_time_week.append(result)
            data["open_time"] = sort_by_weekday(open_time_week)
        except:
            open_time = []
            data["open_time"] = open_time
        try:
            library = driver.find_element(By.CSS_SELECTOR,'.aoRNLd.kn2E5e.NMjTrf.lvtCsd')
            library.click()
            time.sleep(0.5)
            scrollPage(driver,3)
            time.sleep(0.5)
            data["photo"] = get_photo_library(driver)
        except:
            data["photo"] = []
    except:
        data["subtitle"] = ""
        data["info"] = ""
        data["site"] = ""
        data["open_time"] = []
        data["photo"] = []
    return data

def get_photo(style):
    url_match = re.search(r'url\((.+)\);', style)
    if url_match:
        extracted_url = url_match.group(1)
        return extracted_url
    
def multiply_wh_parameters(original_url, factor):
    def modify_size(match):
        return f"{match.group(1)}{int(match.group(2)) * factor}"
    
    url_parts = original_url.split('=')

    if len(url_parts) > 1:
        modified_params = re.sub(r'([wh])(\d+)', modify_size, url_parts[1])
        modified_url = url_parts[0] + '=' + modified_params
    else:
        modified_url = original_url
    
    return modified_url

def get_photo_library(driver):
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    style_element = soup.find_all('div', class_='U39Pmb')

    photo_library = []

    for element in style_element:
        style_attribute = element['style']
        url = get_photo(style_attribute)
        url_pattern = re.compile(r'https?://(?:www\.)?[\w\.-]+(?:\.[a-zA-Z]{2,6})+[\w\.-]*[\?#]?[\w\.-=&]*')
        url = url.replace('"',"")
        if re.match(url_pattern, url):
            photo_library.append(multiply_wh_parameters(url,10))

    return photo_library

def get_time_list(driver):
    open_time_week = []

    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')

    elements = soup.select('.OqCZI.fontBodyMedium.VrynGf')
    elements = elements[0]
    elements = elements.select('.y0skZc')

    for time_up in elements:
        week = time_up.select('td div')[0].text
        clock = time_up.select('td.mxowUb .G8aQO')[0].text
        result={}
        result['week'] = week
        result['clock'] = clock
        open_time_week.append(result)

    timeList = sort_by_weekday(open_time_week)
    return timeList

In [3]:
name = '駁二藝術特區'
url = 'https://www.google.com/maps/place/%E9%A7%81%E4%BA%8C%E8%97%9D%E8%A1%93%E7%89%B9%E5%8D%80/@22.6199803,120.2815287,17z/data=!3m1!4b1!4m6!3m5!1s0x346e047620e1c45b:0xe6d6d1bd1c764df2!8m2!3d22.6199803!4d120.2815287!16s%2Fm%2F010glgxt?entry=ttu'

In [83]:
driver = webdriver.Chrome()
driver.set_window_size(800,800)
driver.get(url)

time.sleep(2)
data = get_place_more_info(driver)
print(data)

attraction_db.attractionInfo.update_one(
    {'name': name},
    {'$set': {
        'subtitle': data['subtitle'],
        'info': data['info'],
        'site': data['site'],
        'open_time': data['open_time'],
        'photo': data['photo']
    }}
) 

{'subtitle': '景點', 'info': '這個碼頭旁的熱鬧廣場設有攤販，可欣賞河川景觀，步道在入夜後會燈火通明。', 'site': 'https://business.google.com/create?fp=970116378257390920&hl=zh-TW&authuser=0&gmbsrc=tw-zh-tw-et-ip-z-gmb-s-z-l~mrc%7Cclaimbz%7Cu&ppsrc=GMBMI&utm_campaign=tw-zh-tw-et-ip-z-gmb-s-z-l~mrc%7Cclaimbz%7Cu&utm_source=gmb_mrc81&utm_medium=et&getstarted&lis=1', 'open_time': [{'week': '星期一', 'clock': '24 小時營業'}, {'week': '星期二', 'clock': '24 小時營業'}, {'week': '星期三', 'clock': '24 小時營業'}, {'week': '星期四', 'clock': '24 小時營業'}, {'week': '星期五', 'clock': '24 小時營業'}, {'week': '星期六', 'clock': '24 小時營業'}, {'week': '星期日', 'clock': '24 小時營業'}], 'photo': ['https://lh5.googleusercontent.com/p/AF1QipM-eYt7B537nqB7NCtgbtzi5JV4Q6GY5YpG_SsV=w2030-h1520-k-no', 'https://lh5.googleusercontent.com/p/AF1QipNK4TIFtq2-k1TKNCPqXLURzazOiXqHhs2qTLe0=w2030-h1520-k-no', 'https://lh5.googleusercontent.com/p/AF1QipP2zroQiXBXajP4Qm8hQUmUECRDzm7jIRDSKZNK=w2030-h1520-k-no', 'https://lh5.googleusercontent.com/p/AF1QipPy647Sqd6pBwR8ZrDF_xRtshzl8ZsBzZoJGa1g=w20

In [6]:
data_list = attraction_db.attractionInfo.find({"open_time": []})

for data in data_list:
    driver = webdriver.Chrome()
    driver.set_window_size(800,800)
    driver.get(data['google_url'])
    time.sleep(1)
    height = 200
    counter = 0 

    while counter  < 10:
        counter += 1

        if counter == 9: break
        try:
            more = driver.find_element(By.CSS_SELECTOR,'.RcCsl.fVHpi.w4vB1d.NOE9ve.AG25L.lk2Rcf button.CsEnBe')
            more.click()
        except:
            pane = driver.find_element("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]')
            driver.execute_script(f"arguments[0].scrollTop = {height}", pane)
            height = height + 200

    time.sleep(1)
    try:
        timeList = get_time_list(driver)
        print(timeList)
                    
        attraction_db.attractionInfo.update_one(
            {'id': data['id']},
            {'$set': {
                'open_time': timeList
            }}
        ) 

        print(f"{data['name']} ：已更新")
    except:
        print(f"{data['name']} ：未更新")
        driver.quit()
        continue
    
    driver.quit()

武荖坑風景區 ：未更新
合歡山 ：未更新
大屯遊憩區 ：未更新
臺中國家歌劇院 ：未更新
墾丁海水浴場 ：未更新
蘭嶼 ：未更新
鐵砧山 ：未更新
[{'week': '星期一', 'clock': '10:00–18:00'}, {'week': '星期二', 'clock': '10:00–18:00'}, {'week': '星期三', 'clock': '10:00–18:00'}, {'week': '星期四', 'clock': '10:00–18:00'}, {'week': '星期五', 'clock': '10:00–20:00'}, {'week': '星期六', 'clock': '10:00–20:00'}, {'week': '星期日', 'clock': '10:00–20:00'}]
駁二藝術特區 ：已更新
[{'week': '星期一', 'clock': '休息'}, {'week': '星期二', 'clock': '09:00–17:00'}, {'week': '星期三', 'clock': '09:00–17:00'}, {'week': '星期四', 'clock': '09:00–17:00'}, {'week': '星期五', 'clock': '09:00–17:00'}, {'week': '星期六', 'clock': '09:00–17:00'}, {'week': '星期日', 'clock': '09:00–17:00'}]
國立科學工藝博物館 ：已更新
愛河 ：未更新
基隆嶼 ：未更新
日月潭環潭區 ：未更新
[{'week': '星期一', 'clock': '休息'}, {'week': '星期二', 'clock': '休息'}, {'week': '星期三', 'clock': '休息'}, {'week': '星期四', 'clock': '休息'}, {'week': '星期五', 'clock': '09:00–17:00'}, {'week': '星期六', 'clock': '09:00–17:00'}, {'week': '星期日', 'clock': '09:00–17:00'}]
國立臺灣歷史博物館 ：已更新
虎頭埤風景區 ：未更新
馬沙溝濱海遊憩區 ：未更新
[{'week': 

CursorNotFound: cursor id 2372582789803786760 not found, full error: {'ok': 0.0, 'errmsg': 'cursor id 2372582789803786760 not found', 'code': 43, 'codeName': 'CursorNotFound', '$clusterTime': {'clusterTime': Timestamp(1693567084, 21), 'signature': {'hash': b':\x80\xd2\xd9m#\xf0\xcci\x94\xe5\xaa\xe8\xbf\x8f%\xf6\xb3\xcc3', 'keyId': 7230861211305246721}}, 'operationTime': Timestamp(1693567084, 21)}

In [15]:
# 抓取詳細景點資訊

name = '南門町三二三 - 台北植物園'
url = 'https://www.google.com/maps/place/%E5%8D%97%E9%96%80%E7%94%BA%E4%B8%89%E4%BA%8C%E4%B8%89+-+%E5%8F%B0%E5%8C%97%E6%A4%8D%E7%89%A9%E5%9C%92/@25.0325241,121.5111008,17z/data=!3m1!4b1!4m6!3m5!1s0x3442a9a37b3ae2e3:0x4c14a8fb08168c03!8m2!3d25.0325241!4d121.5111008!16s%2Fg%2F11fzd1dd19?authuser=0&entry=ttu'

driver = webdriver.Chrome()
driver.set_window_size(800,800)
driver.get(url)

wait = WebDriverWait(driver, 10) 

height = 300
counter = 0 

while counter  < 10:
    counter += 1
    try:
        more = driver.find_element(By.CSS_SELECTOR,'.RcCsl.fVHpi.w4vB1d.NOE9ve.AG25L.lk2Rcf button.CsEnBe')
        more.click()
    except:
        pane = driver.find_element("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]')
        driver.execute_script(f"arguments[0].scrollTop = {height}", pane)
        height = height + 300

time.sleep(1)
timeList = get_time_list(driver)
print(timeList)
        
attraction_db.attractionInfo.update_one(
    {'name': name},
    {'$set': {
        'open_time': timeList
    }}
) 
print('已更新')

driver.quit()

[{'week': '星期一', 'clock': '休息'}, {'week': '星期二', 'clock': '休息'}, {'week': '星期三', 'clock': '09:30–16:30'}, {'week': '星期四', 'clock': '09:30–16:30'}, {'week': '星期五', 'clock': '09:30–16:30'}, {'week': '星期六', 'clock': '09:30–16:30'}, {'week': '星期日', 'clock': '09:30–16:30'}]
已更新


In [16]:
# 檢查景點 site 屬性刪除誤抓的連結，並更新

# 查詢具有符合條件的文檔，並更新
query = {"site": {"$regex": "^https://business.google.com/create"}}
update = {"$set": {"site": ""}}  # 更新 site 屬性為空字串

result = attraction_db.attractionInfo.update_many(query, update)

# 打印更新的文檔數量
print(f"Updated {result.modified_count} documents")

Updated 1228 documents


In [19]:
# 查詢所有文檔，並擷取需要的數據
# data = attraction_db.attractionInfo.find({}, {"name": 1, "rating": 1})
data = attraction_db.attractionInfo.find({})

df = pd.DataFrame(list(data))
df = df.drop(columns=["_id"])

# 指定 CSV 檔案名稱
csv_file = "spotDict.csv"

# 使用 Pandas 將 DataFrame 寫入 CSV 檔案
df.to_csv(csv_file, index=False, encoding="utf-8-sig")

In [86]:
# 檢查景點id是否唯一

from collections import Counter

# 取得所有的 _id 值
id_values = [doc["_id"] for doc in attraction_db.attractionInfo.find({}, {"id": 1})]

# 使用 Counter 來計算每個 _id 值的出現次數
id_counter = Counter(id_values)

# 找出重複的 _id 值
duplicates = [id_val for id_val, count in id_counter.items() if count > 1]

if duplicates:
    print("有重複的 _id 值:")
    for duplicate in duplicates:
        print(duplicate)
else:
    print("所有的 _id 值都是獨立的")


所有的 _id 值都是獨立的
